In [9]:
import os 
import pickle as pkl
import pandas as pd
import numpy  as np

import matplotlib.pyplot as plt

from sklearn.metrics import r2_score
from autoemulate.compare import AutoEmulate

from sklearn.model_selection import KFold

from sklearn.preprocessing import RobustScaler

def main():
    

    path_X  = '/Users/pmzff/Documents/GitHub/ModularCircFF/Emulation/Input/input_100.csv'
    path_y = '/Users/pmzff/Documents/GitHub/ModularCircFF/Emulation/Outputs/Out_fixed/PCA/PC1.csv'
    
 
    X_df  = pd.read_csv(path_X)
    y_df  = pd.read_csv(path_y)
    #  Select relevant inputs only
    relevant_columns = []
    for col in X_df.columns:
        relevant_columns.append(col)
        if col == 'T': break

    # Select only first 5 inputs 
    filtered_input = X_df[relevant_columns]

    filtered_input

   

        
    for col in y_df.columns: 
        if col == 'dt': continue   
        print('\n'*2)
        print(f'Col {col}\n')
        ae = AutoEmulate()
        model_subset = [
            "NeuralNetSk", 
            "SecondOrderPolynomial", 
            # "SupportVectorMachines", 
            # "GradientBoosting", 
            # "RandomForest"
        ]
        ae.setup(filtered_input .values, 
                y_df[col].values, 
                models=model_subset, 
                n_jobs=1, 
                param_search=True,
                scale=RobustScaler(),
                param_search_type='random',  # bayes, random
                param_search_iters=20,
                cross_validator=KFold(n_splits=5, shuffle=True),
                log_to_file=True
                )
        best_model = ae.compare() 
        
        print(ae.summarise_cv())
        
        best_emulator = ae.refit(best_model) 
        
        print(best_emulator.named_steps['model'].get_params())
        
        # ae.save(best_emulator, os.path.join(path_io, 'Emulators', f'emulator_{N_cases}_dt_{dt}_feat_{col}.pkl'))
    
    return



In [10]:
main()




Col PC1



,Values
Simulation input shape (X),"(100, 5)"
Simulation output shape (y),"(100,)"
Proportion of data for testing (test_set_size),0.2
Scale input data (scale),RobustScaler()
Scaler (scaler),StandardScaler
Do hyperparameter search (param_search),True
Type of hyperparameter search (search_type),random
Number of sampled parameter settings (param_search_iters),20
Reduce dimensionality (reduce_dim),False
Cross validator (cross_validator),KFold


Initializing:   0%|          | 0/2 [00:00<?, ?it/s]

                   model short      rmse        r2
0  SecondOrderPolynomial   sop  0.630045  0.994756
1            NeuralNetSk   nns  0.812828  0.990874
{'degree': 2}
